# ML model for generating text based on Wikipedia articles

In [55]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [56]:
with open('../data/text.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

Remove newline characters

In [73]:
corpus = [d[:-1] for d in data]

In [58]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [59]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [60]:
input_sequences

[[1, 187],
 [1, 187, 2],
 [1, 187, 2, 47],
 [1, 187, 2, 47, 8],
 [1, 187, 2, 47, 8, 172],
 [1, 187, 2, 47, 8, 172, 5],
 [1, 187, 2, 47, 8, 172, 5, 16],
 [1, 187, 2, 47, 8, 172, 5, 16, 950],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3, 1],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3, 1, 740],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3, 1, 740, 951],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3, 1, 740, 951, 18],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3, 1, 740, 951, 18, 99],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3, 1, 740, 951, 18, 99, 741],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3, 1, 740, 951, 18, 99, 741, 742],
 [1, 187, 2, 47, 8, 172, 5, 16, 950, 3, 1, 740, 951, 18, 99, 741, 742, 743],
 [1,
  187,
  2,
  47,
  8,
  172,
  5,
  16,
  950,
  3,
  1,
  740,
  951,
  18,
  99,
  741,
  742,
  743,
  148],
 [1,
  187,
  2,
  47,
  8,
  172,
  5,
  16,
  950,
  3,
  1,
  740,
  951,
  18,
  99,
  741,
  742,
  743,
  148,
  45],
 [1,
  187,
  2,
  47,
  8,
  172,
  5,
  16

pre padding so that the last index in each input sequence can be used as the label and all other values are the features

In [61]:
max_sequence = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence, padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  187],
       [   0,    0,    0, ...,    1,  187,    2],
       [   0,    0,    0, ...,  187,    2,   47],
       ...,
       [   0,    0,    0, ...,    1, 4359, 4360],
       [   0,    0,    0, ..., 4359, 4360,    1],
       [   0,    0,    0, ..., 4360,    1,  562]])

In [62]:
# everything but the last word in a sequence
xs = input_sequences[:,:-1]

# labels are the last word in each sequence
labels = input_sequences[:,-1:]

In [63]:
# one hot encode the labels
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)
ys

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [68]:
model = Sequential()

# sub 1 from max_sequence_length because we removed the last index to use as the label
model.add(Embedding(total_words, 100, input_length=max_sequence-1))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [69]:
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
663/663 [==============================] - 74s 112ms/step - loss: 6.9494 - accuracy: 0.0747
Epoch 2/100
663/663 [==============================] - 78s 117ms/step - loss: 6.5588 - accuracy: 0.0791
Epoch 3/100
663/663 [==============================] - 78s 117ms/step - loss: 6.3738 - accuracy: 0.0913
Epoch 4/100
663/663 [==============================] - 80s 120ms/step - loss: 6.2572 - accuracy: 0.1006
Epoch 5/100
663/663 [==============================] - 79s 119ms/step - loss: 6.1476 - accuracy: 0.1029
Epoch 6/100
663/663 [==============================] - 79s 119ms/step - loss: 6.0566 - accuracy: 0.1082
Epoch 7/100
663/663 [==============================] - 79s 119ms/step - loss: 5.9893 - accuracy: 0.1096
Epoch 8/100
663/663 [==============================] - 78s 118ms/step - loss: 5.9319 - accuracy: 0.1120
Epoch 9/100
663/663 [==============================] - 78s 118ms/step - loss: 5.8831 - accuracy: 0.1159
Epoch 10/100
663/663 [==============================] - 78s 118m

663/663 [==============================] - 78s 118ms/step - loss: 5.3916 - accuracy: 0.1505
Epoch 80/100
663/663 [==============================] - 79s 119ms/step - loss: 5.4054 - accuracy: 0.1516
Epoch 81/100
663/663 [==============================] - 78s 118ms/step - loss: 5.4075 - accuracy: 0.1523
Epoch 82/100
663/663 [==============================] - 78s 117ms/step - loss: 5.4116 - accuracy: 0.1502
Epoch 83/100
663/663 [==============================] - 79s 118ms/step - loss: 5.4262 - accuracy: 0.1512
Epoch 84/100
663/663 [==============================] - 82s 123ms/step - loss: 5.4106 - accuracy: 0.1522
Epoch 85/100
663/663 [==============================] - 80s 121ms/step - loss: 5.4194 - accuracy: 0.1529
Epoch 86/100
663/663 [==============================] - 78s 117ms/step - loss: 5.4315 - accuracy: 0.1543
Epoch 87/100
663/663 [==============================] - 78s 117ms/step - loss: 5.4230 - accuracy: 0.1552
Epoch 88/100
663/663 [==============================] - 77s 116ms/st

In [70]:
test_sent = 'began'

In [71]:
for i in range(500):
    test_token_list = tokenizer.texts_to_sequences([test_sent])[0]
    test_sequence = np.array(pad_sequences([test_token_list], maxlen=max_sequence - 1, padding='pre'))
    result = model.predict_classes(test_sequence)
    
    for word, index in tokenizer.word_index.items():
        if index == result:
            test_sent += ' ' + word
            break

In [72]:
test_sent

'began to be not the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and the sun and be been not the not been been not not been a second a way and be the ear been not the sun in least not a not been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been been a way of not and not is the sun and not the not been been the sun is not is not not the not the not the sun in not is the sun in not is the sun in not i